In [41]:
import boa
import sys, os
from eth_utils import to_checksum_address
from boa.contracts.abi.abi_contract import ABIContractFactory
from eth_account.account import Account, LocalAccount

parent_dir = os.path.abspath('..')
sys.path.insert(0, parent_dir)

dotenv_path = os.path.expanduser('~/.web3env') # Path to .env file containing rpcs and keys

def load_contract(contract_address, name=None, api_key=os.environ.get("ETHERSCAN_API_KEY")):
    addr = to_checksum_address(contract_address)
    abi = boa.explorer.fetch_abi_from_etherscan(addr, api_key=api_key)
    return ABIContractFactory.from_abi_dict(abi, name=name).at(addr)

rpc_url = os.getenv("ETH_RPC_URL")
assert rpc_url is not None, "Provider url is not set"
# fork the chain
boa.env.fork(url=rpc_url, block_identifier=20420069)
boa.env.enable_fast_mode()
print(f'Forked the chain on block {boa.env.evm.vm.state.block_number}')

addresses_dict = {
    # "pufETH":       '0xd9a442856c234a39a81a089c06451ebaa4306a72', # does not have version()
    "pufETHwstE":   '0xeeda34a377dd0ca676b9511ee1324974fa8d980d',
    "crvUSD":       '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'
}

topholders = {
    "pufETH":       '0xF047ab4c75cebf0eB9ed34Ae2c186f3611aEAfa6',
    "pufETHwstE":   '0xf4FA0C7833e778fB9FB392eC36217e17C9133976',
    "crvUSD":       '0xA920De414eA4Ab66b97dA1bFE9e6EcA7d4219635'
}

spender = '0xce6431d21e3fb1036ce9973a3312368ed96f5ce7' # some unrelated pool (FRAXsDAI)
spender_sc = load_contract(spender, name='FRAXsDAI')

# Convert addresses to checksum format
for key in addresses_dict.keys():
    addresses_dict[key] = to_checksum_address(addresses_dict[key])
# Fetch contract data from Etherscan and create the contracts dictionary

token_contracts = {key: load_contract(address, name=key) for key, address in addresses_dict.items()}

boa.env.set_random_seed(420)
alice = Account.create()
bob = boa.env.generate_address()
charlie = boa.env.generate_address()

# Give Alice eth and tokens
boa.env.set_balance(alice, boa.env.get_balance(boa.util.abi.Address(alice.address)) + 1*10**18)
for token in addresses_dict.keys():
    with boa.env.prank(topholders[token]):
        token_contracts[token].transfer(alice.address, 1*10**18)
    print(f'Alice has {token_contracts[token].balanceOf(alice.address)} {token}')


Forked the chain on block 20420069


TypeError: Unsupported type: '<class 'eth_account.signers.local.LocalAccount'>'. Must be one of: bool, str, bytes, bytearray or int.

In [15]:
token_contracts['crvUSD'].version()

'v1.0.0'

In [16]:
# firsts check that allowance is 0
for token in addresses_dict.keys():
    assert token_contracts[token].allowance(alice, bob) == 0
    print(f'Allowance for {bob} is 0 for {token}')

Allowance for 0x108b16674BBe7818f37091c31B37E7A911208AAD is 0 for pufETHwstE
Allowance for 0x108b16674BBe7818f37091c31B37E7A911208AAD is 0 for crvUSD


In [37]:
from eip712 import EIP712Message
from eth_account._utils.signing import to_bytes32


def permit_class(contract) -> type[EIP712Message]:
    class Permit(EIP712Message):
        # EIP-712 Domain Fields
        _name_: "string" = contract.name()
        _version_: "string" = contract.version()
        _chainId_: "uint256" = boa.env.evm.chain.chain_id
        _verifyingContract_: "address" = contract.address
        _salt_: "bytes32" = contract.salt()

        # EIP-2612 Data Fields
        owner: "address"
        spender: "address"
        value: "uint256"
        nonce: "uint256"
        deadline: "uint256"

    return Permit

# now try to issue permit so that Bob can transfer Alice's tokens
for token in addresses_dict.keys():
    print(f'Permitting {bob} to spend {token} ({addresses_dict[token]}) on behalf of {alice}')
    # first generate signatiure from alice
    value = 2**256 - 1
    permit = permit_class(token_contracts[token])(owner=alice, spender=bob, value=value, nonce=0, deadline=2**256 - 1)
    # sig = eth_acc.sign_message(permit.signable_message)
    print(permit.signable_message)


Permitting 0x108b16674BBe7818f37091c31B37E7A911208AAD to spend pufETHwstE (0xEEda34A377dD0ca676b9511EE1324974fA8d980D) on behalf of 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945
SignableMessage(version=HexBytes('0x01'), header=HexBytes('0x4930e62b65934f63c2278e52ff3499d854dee334007b6f38e1ca7828ac12eb5b'), body=HexBytes('0x5a051390f6f430cfc7da793c812bc082a51558454cf9dbb8c1d0e955079c76ea'))
Permitting 0x108b16674BBe7818f37091c31B37E7A911208AAD to spend crvUSD (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E) on behalf of 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945
SignableMessage(version=HexBytes('0x01'), header=HexBytes('0x7906987b5141f4b3a52bd45bf8cd1495b428ff72f3caefd1d670d9726103a7f0'), body=HexBytes('0x5a051390f6f430cfc7da793c812bc082a51558454cf9dbb8c1d0e955079c76ea'))


In [43]:
a = Account.create()
a.address

'0x42c14f32868cC6A0Ee2e628ab4C1247Dd006Fe8b'

In [48]:
boa.env.generate_address()

Address('0x849229Fb812e736ee264A84475beaf1F4A931cE4')

In [50]:
boa.util.abi.Address(a.address)

Address('0x42c14f32868cC6A0Ee2e628ab4C1247Dd006Fe8b')